In [1]:
from pathlib import Path
import pandas as pd
import seizure_data_processing as sdp
from src.config import TUSZ_DIR
from seizure_data_processing import EEG

In [2]:
# keywords

LEFT_TEMPORAL = "left temporal"
RIGHT_TEMPORAL = "right temporal"

In [3]:
# only LE reference
LE_TRAIN_FOLDER = TUSZ_DIR + "edf/train/02_tcp_le/"
LE_DEV_FOLDER = TUSZ_DIR + "edf/dev/02_tcp_le/"

FOLDERS = [LE_TRAIN_FOLDER, LE_DEV_FOLDER]

In [4]:
files = []
for folder in FOLDERS:
    files = files + list(Path(folder).rglob("*.[tT][xX][tT]"))

In [5]:
# files[0].parts[6:.replace("\\", "/")
# int(files[10].parts[-1].split('_')[0])

In [6]:
# import os
# temp = []
# for root, dirs, files in os.walk(LE_TRAIN_FOLDER):
#     for file in files:
#         if file.endswith(".txt"):
#             temp.append(file)

In [7]:
df_files = pd.DataFrame(columns=["file", "side", "patient"])

In [8]:
for file in files:
    with open(file, "r", encoding="utf-8", errors="ignore") as f:
        temporal = False
        left = False
        right = False
        lines = f.readlines()
        for line in lines:
            if LEFT_TEMPORAL in line:
                left = True
            if RIGHT_TEMPORAL in line:
                right = True
        if left:
            side = "left"
        elif right:
            side = "right"

    if left or right:
        patient = int(file.parts[-1].split("_")[0])
        file = str(Path(*file.parts[6:])).replace("\\", "/")
        df_files = df_files.append(
            {"file": file, "side": side, "patient": patient}, ignore_index=True
        )

C:\Users\selinederooij\AppData\Local\Temp\ipykernel_14388\110358461.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_files = df_files.append({'file': file, 'side': side, 'patient': patient}, ignore_index=True)
C:\Users\selinederooij\AppData\Local\Temp\ipykernel_14388\110358461.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_files = df_files.append({'file': file, 'side': side, 'patient': patient}, ignore_index=True)
C:\Users\selinederooij\AppData\Local\Temp\ipykernel_14388\110358461.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_files = df_files.append({'file': file, 'side': side, 'patient': patient}, ignore_index=True)
C:\Users\selinederooij\AppData\Local\Temp\ipykernel_14388\110358461.py:20: FutureW

In [9]:
df_files = df_files.drop_duplicates(subset=["patient"])

Merge with patient info (No. Seizure per Patient)

In [10]:
seiz_ann = pd.read_csv("../data/Seizure_Annotations.csv")
seiz_ann = seiz_ann.dropna(subset=["Seizure_Type"])

In [11]:
df_files["num_seizures"] = 0
for patient in df_files["patient"]:
    temp = seiz_ann.loc[seiz_ann["Patient"] == patient]
    temp = temp.loc[
        temp["Seizure_Stop"] - temp["Seizure_Start"] > 10.0
    ]  # only seizures longer than 10 s
    num_seiz = len(temp)
    df_files.loc[df_files["patient"] == patient, "num_seizures"] = num_seiz

In [12]:
df_files.to_csv("../data/TLE_patients.csv", index=True)

Now check for T1 and T2 channels in the patients' files

In [13]:
patient_ann = pd.read_csv("../data/Seizure_Annotations.csv")
patient_ann = patient_ann.loc[patient_ann["Patient"].isin(df_files["patient"].tolist())]
patient_ann["T1_T2"] = False
for i, row in patient_ann.iterrows():
    file = TUSZ_DIR + "edf" + row["Filename"] + ".edf"
    eeg = EEG(file)
    index = [
        i for i, elem in enumerate(eeg.channels) if "T1".casefold() in elem.casefold()
    ]
    if len(index) > 0:
        patient_ann.at[i, "T1_T2"] = True

C:\Users\selinederooij\surfdrive\Code\data_processing_library\seizure_data_processing\seizure_data_processing\pre_processing\eeg.py:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.data = np.array(signals)
C:\Users\selinederooij\surfdrive\Code\data_processing_library\seizure_data_processing\seizure_data_processing\pre_processing\eeg.py:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.data = np.array(signals)
C:\Users\selinederooij\surfdrive\Code\data_processing_library\seizure_data_processing\seizure_data_processing\pre_processing\eeg.py:

In [14]:
patient_ann = patient_ann.loc[patient_ann["T1_T2"] == True]

In [15]:
patients = patient_ann["Patient"].unique().tolist()
df_files = df_files.loc[df_files["patient"].isin(patients)]
for patient in df_files["patient"]:
    temp = patient_ann.loc[patient_ann["Patient"] == patient]
    temp = temp.loc[
        temp["Seizure_Stop"] - temp["Seizure_Start"] > 10.0
    ]  # only seizures longer than 10 s
    num_seiz = len(temp)
    df_files.loc[df_files["patient"] == patient, "num_seizures"] = num_seiz

df_files.to_csv("../data/TLE_patients.csv", index=True)

In [16]:
patient_ann.to_csv("../data/annotations_TLE_patients.csv", index=False)